In [25]:
import numpy as np
# Define Pauli operations using PauliTerm
# Defining the Pauli matrices
I = np.array([[1.+0.j, 0.+0.j],
            [0.+0.j, 1.+0.j]])

X = np.array([[0.+0.j, 1.+0.j],
            [1.+0.j, 0.+0.j]])

Y = np.array([[0.+1.j, 0.+0.j],
            [0.+0.j, 0.-1.j]])

Z = np.array([[1.+0.j, 0.+0.j],
            [0.+0.j, -1.+0.j]])

cnot = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 1],
                [0, 0, 1, 0]])

# Pauli products
pauli_YZ = np.kron(Y, Z)
pauli_IY = np.kron(I, Y)
pauli_YY = np.kron(Y, Y)
pauli_XY = np.kron(X, Y)
pauli_IX = np.kron(I, X)
pauli_II = np.kron(I, I)

print(np.matmul(cnot,pauli_II))

print(np.kron(np.array([[1]]), I))

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


In [26]:
def apply_noise_model(prog, backend, error_template, gate):
	import numpy as np
	# Define Pauli operations using PauliTerm
	# Defining the Pauli matrices
	Paulis = {}
	Paulis['I'] = np.array([[1.+0.j, 0.+0.j],
				[0.+0.j, 1.+0.j]])

	Paulis['X'] = np.array([[0.+0.j, 1.+0.j],
				[1.+0.j, 0.+0.j]])

	Paulis['Y'] = np.array([[0.+1.j, 0.+0.j],
				[0.+0.j, 0.-1.j]])

	Paulis['Z'] = np.array([[1.+0.j, 0.+0.j],
				[0.+0.j, -1.+0.j]])

	Gates = {}
	Gates['CNOT'] = np.array([[1, 0, 0, 0],
					[0, 1, 0, 0],
					[0, 0, 0, 1],
					[0, 0, 1, 0]])

	noise_model = []

	for op, p in error_template:
		errorgate = np.array([[1]])
		for char in op:
			errorgate = np.kron(errorgate, Paulis[char])
		noise_model.append(np.sqrt(p)*np.matmul(errorgate, Gates[gate]))


	if np.log2(np.linalg.matrix_rank(errorgate)) == 2:
		for i, j in backend.qubit_topology().edges:
			prog.define_noisy_gate(gate, [i,j], noise_model)
			prog.define_noisy_gate(gate, [j,i], noise_model)
	elif np.log2(np.linalg.matrix_rank(errorgate)) == 1:
		for i in backend.qubits():
			prog.define_noisy_gate(gate, [i], noise_model)

In [27]:
from TrotterExample import executor, get_backend
from pyquil import Program
from pyquil.gates import H, CNOT, Z, MEASURE, S, X, Y, I, RX, RZ, FENCE
from pyquil.quilbase import Declare
from pyquil.api import local_forest_runtime
import numpy as np

with local_forest_runtime():

	# Here the backend for the simulation is prepared
	from pyquil.quantum_processor import NxQuantumProcessor
	from pyquil.noise import NoiseModel
	from pyquil import get_qc
	import networkx as nx
	backend = get_qc("5q-qvm") #str(n)+'q-qvm' #args.backend
	isa = backend.to_compiler_isa()
	backend_qubits = sorted(int(k) for k in isa.qubits.keys())
	# By default every qubit on our fake backend is connected with every other qubit. This breaks the algorhythm. So we need to change the topology
	# So this line here changes the topology of the fake backend. The condition in the end determines it
	edges = [(q1, q2) for q1 in backend_qubits for q2 in backend_qubits if abs(q1-q2) == 1]
	# Build the NX graph
	topo = nx.from_edgelist(edges)
	# You would uncomment the next line if you have disconnected qubits
	# topo.add_nodes_from(qubits)
	nx_quantum_processor = NxQuantumProcessor(topo)
	backend.compiler.quantum_processor = nx_quantum_processor

	prog = Program()
	prog += Declare("ro", "BIT", 2)
	prog += Z(0)
	prog += CNOT(0,1)
	prog += Z(1)

	prog2 = Program()
	prog2 += H(1)
	prog2 += H(0)
	prog2 = prog2.dagger()
	prog2 = backend.compile(prog2)
	print(prog2)

	prog += prog2
	prog += X(0)
	prog += MEASURE(0, ("ro", 0))
	prog += MEASURE(1, ("ro", 1))
	flip =  np.array([[0.+0.j, 1.+0.j],
							[1.+0.j, 0.+0.j]])
	test = [np.array([[1.        , 0.        ],
		[0.        , 0.98994949]]), np.array([[0.        , 0.14142136],
		[0.        , 0.        ]])]


	twoqubit_errorops = ['IX']
	twoqubit_errorprobs = [0.05]
	twoqubit_error_template = [(op, p) for op,p in zip(twoqubit_errorops, twoqubit_errorprobs)]+[("II", 1-sum(twoqubit_errorprobs))]
	#(noise_model, twoqubit_error_template, singlequbit_error_template) = get_noise_model()
	#prog.define_noisy_gate("X", [0], [identity])
	def make_prog_noisy(prog):
		prog.define_noisy_gate("CNOT", [0,1], [np.sqrt(0.5)*cnot, np.sqrt(0.5)*pauli_II])#append_damping_to_gate(np.eye(2), 0.1))
	#prog.define_noisy_gate("CNOT", [1,0], [pauli_IX])
	apply_noise_model(prog, backend, twoqubit_error_template, "CNOT")

	counts = executor([prog], backend, 1024, apply_noise=False)
	print(counts)
    

RZ(pi) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RZ(pi) 1
RX(pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 1
HALT

DECLARE ro BIT[2]
Z 0
CNOT 0 1
Z 1
RZ(pi) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RZ(pi) 1
RX(pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 1
HALT
X 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]
PRAGMA ADD-KRAUS CNOT 0 1 "(0.0 0.22360679774997896 0.0 0.0 0.22360679774997896 0.0 0.0 0.0 0.0 0.0 0.22360679774997896 0.0 0.0 0.0 0.0 0.22360679774997896)"
PRAGMA ADD-KRAUS CNOT 0 1 "(0.9746794344808963 0.0 0.0 0.0 0.0 0.9746794344808963 0.0 0.0 0.0 0.0 0.0 0.9746794344808963 0.0 0.0 0.9746794344808963 0.0)"
PRAGMA ADD-KRAUS CNOT 1 0 "(0.0 0.22360679774997896 0.0 0.0 0.22360679774997896 0.0 0.0 0.0 0.0 0.0 0.22360679774997896 0.0 0.0 0.0 0.0 0.22360679774997896)"
PRAGMA ADD-KRAUS CNOT 1 0 "(0.9746794344808963 0.0 0.0 0.0 0.0 0.9746794344808963 0.0 0.0 0.0 0.0 0.0 0.9746794344808963 0.0 0.0 0.9746794344808963 0.0)"
PRAGMA ADD-KRAUS CNOT 1 2 "(0.0 0.22360679774997896 0.0 0.0 0.22360679774997896 0.0 0.0 0.0 0.0 0.0 0.22360679774997896 0.